<a href="https://colab.research.google.com/github/sseyonafre/Project/blob/main/AI_14_%EA%B3%A0%EC%84%B8%ED%9D%AC_Section3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()


Saving user.csv to user.csv


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

In [4]:
df = pd.read_csv('user.csv',index_col = 1) 

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16541 entries, 1 to 16541
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        16541 non-null  int64  
 1   ageClass          16541 non-null  int64  
 2   ageDegree         16541 non-null  int64  
 3   ageGbn            16541 non-null  object 
 4   certGbn           16541 non-null  object 
 5   height            16541 non-null  float64
 6   weight            16541 non-null  float64
 7   crunch            16541 non-null  object 
 8   jump              16541 non-null  object 
 9   trunkFlexion      16541 non-null  object 
 10  IllinoisAgility   16541 non-null  object 
 11  BMI               16541 non-null  float64
 12  situp             16541 non-null  object 
 13  standinglongjump  16541 non-null  object 
 14  standsit          16541 non-null  object 
 15  twominwalk        16541 non-null  object 
 16  threeMwalk        16541 non-null  object

# **EDA**

In [6]:
df.drop(columns=['Unnamed: 0'],inplace=True)
df.drop_duplicates(keep='first', inplace=True)
df.replace('-',np.nan,inplace=True)
for i in ['jump','situp','crunch','trunkFlexion','IllinoisAgility','standinglongjump','twominwalk','threeMwalk']:
    df[i]= df[i].astype(float)
df['certGbn'] = df['certGbn'].astype('category')
df.certGbn.replace({'참가증':'4','1등급':'1','2등급':'2','3등급':'3'},inplace=True)

In [7]:
df1 = df.query("ageGbn=='노인'")
df2 = df.query("ageGbn=='성인'")
df3 = df.query("ageGbn=='유소년'")
df4 = df.query("ageGbn=='청소년'")

df1.drop(columns=['crunch','jump','IllinoisAgility','situp','standinglongjump'],inplace=True)
df2.drop(columns=['crunch','jump','IllinoisAgility','standsit','twominwalk','threeMwalk'],inplace=True)
df3.drop(columns=['jump','situp','standsit','twominwalk','threeMwalk'],inplace=True)
df4.drop(columns=['crunch','situp','standsit','twominwalk','threeMwalk'],inplace=True)

df1.standsit = df1.standsit.replace(np.nan,0)
df2.situp = df2.situp.replace(np.nan,0).round(0).astype(int)
df3.crunch = df3.crunch.replace(np.nan,0).round(0).astype(int)
df4.jump = df4.jump.replace(np.nan,0).round(0).astype(int)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


# **모델링**
> ## 나이구분에 따라 검사항목이 다르기 때문에 모델을 나이구분별로 만들어 준다.


### 1. 노인층 모델링
> 기준모델

In [8]:
#노인층의 모델.

In [9]:
#train, test 셋으로 분리 , target 설정.

target = 'certGbn'

train, test = train_test_split(df1, train_size=0.80, test_size=0.20, 
                              stratify=df1[target], random_state=2)
features1 = df1.drop(columns=['certGbn','exercise','testYm','ageGbn']).columns

X_train1 = train[features1]
y_train1 = train[target]
X_test1 = test[features1]
y_test1 = test[target]

In [10]:
#기준모델 
y_train1.value_counts(normalize=True) #불균형. 다중분류.

4    0.513089
3    0.233857
2    0.178010
1    0.075044
Name: certGbn, dtype: float64

In [11]:
#기준모델을 타겟의 최빈값으로 설정.
baseline1 = y_train1.mode()

In [12]:
#기준모델의 정확도를 구해보자.
y_pred = [baseline1] * len(y_test1)
print("기준모델 정확도: ", accuracy_score(y_test1, y_pred))

기준모델 정확도:  0.5138888888888888


> 로지스틱회귀모델

In [13]:
#로지스틱회귀모델을 만들어 교차검증을 수행. #베이지안최적화를 사용해서 최적의 파라미터 찾아보자.

pipe1_lr = make_pipeline(
    SimpleImputer(), 
    StandardScaler(),

    LogisticRegression(n_jobs=-1)
)

k = 5

scores = cross_val_score(pipe1_lr, X_train1, y_train1, cv=k, scoring='f1_weighted')
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.68627044 0.65152084 0.63891139 0.6014879  0.64729652]
교차검증 f1 평균: 0.6450974184243188


> 랜덤포레스트모델

In [14]:
pipe1_rfc = make_pipeline(
    SimpleImputer(), 
    RandomForestClassifier(n_estimators=100, random_state=2, n_jobs=-1)
)


k = 5

scores = cross_val_score(pipe1_rfc, X_train1, y_train1, cv=k, scoring='f1_weighted') #다중분류에서 f1score 사용하려면 weighted
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.73869633 0.7083339  0.65990913 0.67689155 0.67826721]
교차검증 f1 평균: 0.6924196227015713


> XGBoost모델


In [15]:

pipe1_xg = make_pipeline(
    SimpleImputer(), 
    XGBClassifier(random_state=2, n_jobs=-1)
)

pipe1_xg.fit(X_train1,y_train1)

k = 5

scores = cross_val_score(pipe1_xg, X_train1, y_train1, cv=k, 
                         scoring='f1_weighted')
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.76716595 0.66270517 0.6287545  0.69748192 0.67064501]
교차검증 f1 평균: 0.6853505119410261


### 2. 성인층 모델링
> 기준모델

In [16]:
#train, test 셋으로 분리 , target 설정.

train, test = train_test_split(df2, train_size=0.80, test_size=0.20, 
                              stratify=df2[target], random_state=2)
features2 = df2.drop(columns=['certGbn','exercise','testYm','ageGbn']).columns

X_train2 = train[features2]
y_train2 = train[target]
X_test2 = test[features2]
y_test2 = test[target]

In [17]:
#기준모델 
y_train2.value_counts(normalize=True) #불균형. 다중분류.

4    0.515831
3    0.209763
2    0.189314
1    0.085092
Name: certGbn, dtype: float64

In [18]:
#기준모델을 타겟의 최빈값으로 설정.
baseline2 = y_train2.mode()
y_pred = [baseline2] * len(y_test2)
print("기준모델 정확도: ", accuracy_score(y_test2, y_pred))

기준모델 정확도:  0.5158311345646438


> 로지스틱회귀모델

In [19]:
pipe2_lr = make_pipeline(
    SimpleImputer(), 
    StandardScaler(),

    LogisticRegression(n_jobs=-1)
)

k = 5

scores = cross_val_score(pipe2_lr, X_train2, y_train2, cv=k, scoring='f1_weighted')
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.65994698 0.63988645 0.6551629  0.68555629 0.65000298]
교차검증 f1 평균: 0.6581111186094544


> 랜덤포레스트모델

In [20]:
pipe2_rfc = make_pipeline(
    SimpleImputer(), 
    RandomForestClassifier(n_estimators=100, random_state=2, n_jobs=-1)
)


k = 5

scores = cross_val_score(pipe2_rfc, X_train2, y_train2, cv=k, scoring='f1_weighted') #다중분류에서 f1score 사용하려면 weighted
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.74413948 0.74312606 0.74435528 0.74218111 0.7425301 ]
교차검증 f1 평균: 0.74326640517639


> XGBoost모델

In [21]:
pipe2_xg = make_pipeline(
    SimpleImputer(), 
    XGBClassifier(random_state=2, n_jobs=-1)
)

pipe2_xg.fit(X_train2,y_train2)

k = 5

scores = cross_val_score(pipe2_xg, X_train2, y_train2, cv=k, 
                         scoring='f1_weighted')
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.74019231 0.70004947 0.72225359 0.73380215 0.74990397]
교차검증 f1 평균: 0.7292402982604944


### 3. 유소년 모델링
> 기준모델

In [22]:
#train, test 셋으로 분리 , target 설정.
train, test = train_test_split(df3, train_size=0.80, test_size=0.20, 
                              stratify=df3[target], random_state=2)
features3 = df3.drop(columns=['ageClass','certGbn','exercise','testYm','ageGbn']).columns

X_train3 = train[features3]
y_train3 = train[target]
X_test3 = test[features3]
y_test3 = test[target]

y_train3.value_counts(normalize=True)

baseline3 = y_train3.mode()
y_pred = [baseline3] * len(y_test3)
print("기준모델 정확도: ", accuracy_score(y_test3, y_pred))

기준모델 정확도:  0.7620528771384136


> 로지스틱회귀모델

In [23]:
pipe3_lr = make_pipeline(
    SimpleImputer(), 
    StandardScaler(),

    LogisticRegression(n_jobs=-1)
)

k = 5

scores = cross_val_score(pipe3_lr, X_train3, y_train3, cv=k, scoring='f1_weighted')
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.71227163 0.70862495 0.73570925 0.72276772 0.77089056]
교차검증 f1 평균: 0.7300528207738002


> 랜덤포레스트모델

In [24]:
pipe3_rfc = make_pipeline(
    SimpleImputer(), 
    RandomForestClassifier(n_estimators=100, random_state=2, n_jobs=-1)
)


k = 5

scores = cross_val_score(pipe3_rfc, X_train3, y_train3, cv=k, scoring='f1_weighted') #다중분류에서 f1score 사용하려면 weighted
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.77544473 0.81815492 0.82480918 0.80333573 0.82024068]
교차검증 f1 평균: 0.8083970457907632


> XGBoost모델

In [25]:
pipe3_xg = make_pipeline(
    SimpleImputer(), 
    XGBClassifier(random_state=2, n_jobs=-1)
)

pipe3_xg.fit(X_train3,y_train3)

k = 5

scores = cross_val_score(pipe3_xg, X_train3, y_train3, cv=k, 
                         scoring='f1_weighted')
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.79622046 0.82800065 0.8408899  0.81381344 0.82991119]
교차검증 f1 평균: 0.8217671281175036


###4. 청소년 모델링
> 기준모델

In [26]:

train, test = train_test_split(df4, train_size=0.80, test_size=0.20, 
                              stratify=df4[target], random_state=2)
features4 = df4.drop(columns=['ageClass','certGbn','exercise','testYm','ageGbn']).columns

X_train4 = train[features4]
y_train4 = train[target]
X_test4 = test[features4]
y_test4 = test[target]

y_train4.value_counts(normalize=True)

baseline4 = y_train4.mode()
y_pred = [baseline4] * len(y_test4)
print("기준모델 정확도: ", accuracy_score(y_test4, y_pred))

기준모델 정확도:  0.6746031746031746


> 로지스틱회귀모델

In [27]:
pipe4_lr = make_pipeline(
    SimpleImputer(), 
    StandardScaler(),

    LogisticRegression(n_jobs=-1)
)

k = 5

scores = cross_val_score(pipe4_lr, X_train4, y_train4, cv=k, scoring='f1_weighted')
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.64105594 0.64339499 0.63686066 0.6480251  0.63431441]
교차검증 f1 평균: 0.6407302217330173


> 랜덤포레스트모델

In [28]:
pipe4_rfc = make_pipeline(
    SimpleImputer(), 
    RandomForestClassifier(n_estimators=100, random_state=2, n_jobs=-1)
)

pipe4_rfc.fit(X_train4,y_train4)

k = 5

scores = cross_val_score(pipe4_rfc, X_train4, y_train4, cv=k, scoring='f1_weighted') #다중분류에서 f1score 사용하려면 weighted
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.73079886 0.73140659 0.73681567 0.74525525 0.72084894]
교차검증 f1 평균: 0.7330250630936702


> XGBoost모델

In [29]:
pipe4_xg = make_pipeline(
    SimpleImputer(), 
    XGBClassifier(random_state=2, n_jobs=-1)
)


k = 5

scores = cross_val_score(pipe4_xg, X_train4, y_train4, cv=k, 
                         scoring='f1_weighted')
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.73939958 0.71528292 0.73010331 0.72717777 0.71723255]
교차검증 f1 평균: 0.7258392269439307


### 결과
 >유소년모델을 제외한 나머지는 랜덤포레스트로 유소년은 xgboost로 결정.



# 모델 피클파일로 저장

In [30]:
# 해당모델들을 피클라이브러리를 이용하여 피클파일로 저장.

In [31]:
import pickle 
with open('senior.pickle','wb') as fw:
    pickle.dump(pipe1_rfc, fw)

In [32]:
with open('middle.pickle','wb') as fw:
    pickle.dump(pipe2_rfc, fw)

In [33]:
with open('underaged.pickle','wb') as fw:
    pickle.dump(pipe3_xg, fw)

In [34]:
with open('youth.pickle','wb') as fw:
    pickle.dump(pipe4_rfc, fw)